In [ ]:
##Technique to fine tune hyperparameters - using Bayesian Optimization based on Probalistic Values

# Dataset - https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv

# Objective Function - what we want to optimize ( train the model with diff hyper parameters and give score )

# Probalistic Model :
# Acquisition Function :
# BO Loop

Dr. Ernesto Lee Article on the same
https://drlee.io/step-by-step-guide-bayesian-optimization-with-random-forest-fdc6f329db9c

In [1]:
import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('winequality-red.csv', sep=';')
X = data.drop('quality', axis=1)
y = data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

FileNotFoundError: ignored

In [ ]:
def objective_function(params):
    kernel = C(params[0], (1e-3, 1e3)) * RBF(params[1], (1e-2, 1e2))
    gp = GaussianProcessRegressor(kernel=kernel)
    gp.fit(X_train, y_train)
    predictions = gp.predict(X_test)
    return -mean_squared_error(y_test, predictions)

In [ ]:
def acquisition_function(x):
    # Expected Improvement acquisition function
    mu, sigma = gp.predict(x, return_std=True)
    return -(mu + 1.96 * sigma)

In [ ]:
best_params = [1, 1]  # initial guess
for iteration in range(10):  # 10 iterations for demonstration purposes
    res = minimize(fun=objective_function, x0=best_params, bounds=[(1e-3, 1e3), (1e-2, 1e2)])
    best_params = res.x

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessin

In [ ]:
# Load the model with optimized parameters
optimized_kernel = C(best_params[0], (1e-3, 1e3)) * RBF(best_params[1], (1e-2, 1e2))
optimized_gp = GaussianProcessRegressor(kernel=optimized_kernel)
optimized_gp.fit(X_train, y_train)

# Making predictions
predictions = optimized_gp.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


In [ ]:
# Practical Approach

In [8]:
!pip install bayesian-optimization scikit-learn pandas

In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from bayes_opt import BayesianOptimization

In [10]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
X = data.drop('quality', axis=1)
y = data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
def objective(n_estimators, max_depth, min_samples_split, max_features):
    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                  max_depth=int(max_depth),
                                  min_samples_split=int(min_samples_split),
                                  max_features=min(max_features, 0.999),  # Fraction, must be <= 1.0
                                  random_state=42)

    return -1.0 * cross_val_score(model, X_train, y_train, cv=3, scoring="neg_mean_squared_error").mean()

In [12]:
# Bounds for hyperparameters
param_bounds = {
    'n_estimators': (10, 250),
    'max_depth': (1, 50),
    'min_samples_split': (2, 25),
    'max_features': (0.1, 0.999),
}

In [13]:
optimizer = BayesianOptimization(f=objective, pbounds=param_bounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=15)

|   iter    |  target   | max_depth | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------------------
| 1         | 0.3948    | 19.35     | 0.9547    | 18.84     | 153.7     |
| 2         | 0.3985    | 8.645     | 0.2402    | 3.336     | 217.9     |
| 3         | 0.3797    | 30.45     | 0.7366    | 2.473     | 242.8     |
| 4         | 0.3808    | 41.79     | 0.2909    | 6.182     | 54.02     |
| 5         | 0.3886    | 15.91     | 0.5718    | 11.93     | 79.89     |
| 6         | 0.3925    | 8.633     | 0.3423    | 4.297     | 217.6     |
| 7         | 0.4288    | 7.583     | 0.1053    | 2.683     | 218.2     |
| 8         | 0.3924    | 8.95      | 0.6261    | 2.061     | 220.2     |
| 9         | 0.4026    | 6.936     | 0.552     | 2.913     | 219.2     |
| 10        | 0.3996    | 7.72      | 0.4593    | 3.708     | 219.0     |
| 11        | 0.4016    | 6.011     | 0.5897    | 3.48      | 218.0     |
| 12        | 0.4126    | 5.478     | 

In [14]:
best_params = optimizer.max['params']

In [15]:
best_params_formatted = {
    'n_estimators': int(best_params['n_estimators']),
    'max_depth': int(best_params['max_depth']),
    'min_samples_split': int(best_params['min_samples_split']),
    'max_features': best_params['max_features']
}

In [16]:
optimized_rf = RandomForestRegressor(**best_params_formatted, random_state=42)

In [17]:
optimized_rf.fit(X_train, y_train)

RandomForestRegressor(max_depth=4, max_features=0.26880661490436264,
                      min_samples_split=4, n_estimators=216, random_state=42)

In [18]:
score = optimized_rf.score(X_test, y_test)
print(f"Test R^2 Score with Optimized Hyperparameters: {score}")

Test R^2 Score with Optimized Hyperparameters: 0.377858715533007
